In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, Reshape, Flatten, Dense
from tensorflow.keras.layers import Dense,Softmax
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from matplotlib import pyplot as plt
import cv2
import math
import os
import numpy as np

def define_model():

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(128, 128, 1)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(Conv2D(64, (1, 1), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(Conv2D(32, (1, 1), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
    model.add(Conv2D(16, (1, 1), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
    model.add(Conv2D(8, (1, 1), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(8, (1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128,activation="linear"))
    model.add(Reshape((4, 4,8)))
    
    return model

In [ ]:
model = define_model()
model.summary()

In [ ]:
def xywh2minmax(xy, wh):
    xy_min = xy - wh / 2
    xy_max = xy + wh / 2

    return xy_min, xy_max

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [ ]:
def label_img(img_path,label_path):
    
    IMAGES = []
    
    LABELS = []
    
    img_list = os.listdir(img_path)
    
    for image_path in img_list:
        
        # Read image
        img = cv2.imread(img_path + "/" + image_path)
        #print(img_path + "/" + img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_resize = cv2.resize(gray,(128,128))
        gray_resize = np.expand_dims(gray_resize, axis=2)
        IMAGES.append(gray_resize)
        
        # Read Labels
        label_pth = label_path + "/" + image_path.split(".")[0]+".txt"
        
        with open(label_pth) as file:
            lines = file.readlines()
            lines = [line.rstrip() for line in lines]
        
        G_T = np.zeros((4,4,8))
        
        for line in lines:
    
            g_t = [0,0,0,0,0,0,0,0]
    
            p=line.split(" ")
    
            g_t[0] = float(p[1])
            g_t[1] = float(p[2])
            g_t[2] = float(p[3])
            g_t[3] = float(p[4])
            g_t[4] = float(1.0)
    
            if p[0]=="0":
                g_t[5] = float(1.0)
            if p[0]=="1":
                g_t[6] = float(1.0)
            if p[0]=="2":
                g_t[7] = float(1.0)
    
            G_T[math.ceil(4*float(p[1]))-1,math.ceil(4*float(p[2]))-1] = g_t
        
        LABELS.append(G_T)
    
    return IMAGES, LABELS

In [ ]:
from matplotlib import pyplot as plt
import cv2
import math
import os
import numpy as np

IMAGES, LABELS = label_img("images","labels")

In [ ]:
def iou_loss(y_true, y_pred):
    # iou loss for bounding box prediction
    # input must be as [x1, y1, x2, y2]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] - K.transpose(y_true)[0] + 1) * K.abs(K.transpose(y_true)[3] - K.transpose(y_true)[1] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] - K.transpose(y_pred)[0] + 1) * K.abs(K.transpose(y_pred)[3] - K.transpose(y_pred)[1] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2], K.transpose(y_pred)[2])
    overlap_3 = K.minimum(K.transpose(y_true)[3], K.transpose(y_pred)[3])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    # loss for the iou value
    iou_loss = -K.log(iou)

    return iou_loss

In [ ]:
def iou(pred_mins, pred_maxes, true_mins, true_maxes):
    intersect_mins = K.maximum(pred_mins, true_mins)
    intersect_maxes = K.minimum(pred_maxes, true_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    pred_wh = pred_maxes - pred_mins
    true_wh = true_maxes - true_mins
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]
    true_areas = true_wh[..., 0] * true_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores = intersect_areas / union_areas

    return iou_scores

In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf

def my_loss_fn(y_true, y_pred):
    
    ## IOU loss
    
    conf_pred = y_pred[..., 4:5]
    conf_pred = K.sigmoid(conf_pred)
    predict_trust = conf_pred
    #y_pred = tf.cast(y_pred,dtype="float64")
    box_true = y_true[..., :4]
    box_pred = y_pred[..., :4]
    box_pred = K.sigmoid(box_pred)
    
    xy_min_pred, xy_max_pred = xywh2minmax(128*box_pred[..., :2],128*box_pred[..., 2:4])
    xy_min_true, xy_max_true = xywh2minmax(128*box_true[..., :2],128*box_true[..., 2:4])
    #################################################################
    
    response_mask = y_true[..., 5]  # ? * 7 * 7
    response_mask = K.expand_dims(response_mask)  # ? * 7 * 7 * 1
    
    iou_scores = iou(xy_min_pred, xy_max_pred, xy_min_true, xy_max_true)
    iou_scores = K.expand_dims(iou_scores)
    iou_scores = K.expand_dims(iou_scores)
    
    best_ious = K.max(iou_scores, axis=4)  # ? * 7 * 7 * 2
    best_box = K.max(best_ious, axis=3, keepdims=True)  # ? * 7 * 7 * 1

    box_mask = K.cast(best_ious >= best_box, K.dtype(best_ious))  # ? * 7 * 7 * 2
    
    no_object_loss = 0.5 * (1 - box_mask * response_mask) * K.square(0 - predict_trust)
    object_loss = box_mask * response_mask * K.square(1 - predict_trust)
    confidence_loss = no_object_loss + object_loss
    confidence_loss = K.sum(confidence_loss)

    #################################################################
    
    #boxes_pred = K.concatenate((xy_min_pred, xy_max_pred),axis=3)
    #boxes_true = K.concatenate((xy_min_true, xy_max_true),axis=3)
    
    #print(boxes_true)
    #print(boxes_pred)
    
    #print(boxes_pred)
    #print(boxes_true)
    #print(iou_loss(boxes_true,boxes_pred))
    
    #box_loss=K.sum(iou_loss(boxes_true,boxes_pred))
    #print(box_loss)
    
    
    #for i in range(4):
    #    for j in range(4):
    #        box_loss = box_loss + (1-bb_intersection_over_union(boxes_pred[0][i][j],boxes_true[0][i][j]))
    
    
    ###############################################
    
    #conf_true = y_true[..., 4:5]
    #conf_pred = y_pred[..., 4:5]
    #conf_pred = K.sigmoid(conf_pred)
    
    #conf_loss = K.square(conf_true - conf_pred)
    #conf_loss = K.sum(conf_loss)
    #print(conf_loss)
    
    class_true = y_true[..., 5:8]
    class_pred = y_pred[..., 5:8]
    class_pred = K.softmax(class_pred)
    
    class_loss = response_mask * K.square(class_true - class_pred)
    class_loss = K.sum(class_loss)
    
    
    box_mask = K.expand_dims(box_mask)
    response_mask = K.expand_dims(response_mask)
    
    label_xy = 128*box_true[..., :2]
    label_wh = 128*box_true[..., 2:4]
    
    predict_xy = 128*box_pred[..., :2]
    predict_wh = 128*box_pred[..., 2:4]

    box_loss = 5 * box_mask * response_mask * K.square((label_xy - predict_xy) / 128)
    box_loss += 5 * box_mask * response_mask * K.square((K.sqrt(label_wh) - K.sqrt(predict_wh)) / 128)
    box_loss = K.sum(box_loss)
    
    loss = box_loss + confidence_loss + class_loss
    
    return loss

In [ ]:
LABELS[0]

In [ ]:
np.expand

In [ ]:
np.expand_dims(LABELS[0],axis=0).shape

In [ ]:
prediction.shape

In [ ]:
import tensorflow as tf
tf.cast(prediction,dtype="float32")

In [ ]:
import tensorflow as tf

#tf.cast(
#    x, dtype=float64, name=None
#)

y_true = label
y_pred = pred

box_true = y_true[..., :4]
box_pred = y_pred[..., :4]
box_pred = K.sigmoid(box_pred)
    
xy_min_pred, xy_max_pred = xywh2minmax(128*y_pred[..., :2],128*y_pred[..., 2:4])
xy_min_true, xy_max_true = xywh2minmax(128*y_true[..., :2],128*y_true[..., 2:4])
    
boxes_pred = K.concatenate((xy_min_pred, xy_max_pred),axis=3)
boxes_true = K.concatenate((xy_min_true, xy_max_true),axis=3)
    
    #print(boxes_true)
    #print(boxes_pred)
    
    #print(boxes_pred)
    #print(boxes_true)
print(boxes_true)
print(boxes_pred)
K.sum(iou_loss(boxes_true,boxes_pred))

In [ ]:
model.compile(optimizer='adam', loss=my_loss_fn)

In [ ]:
IMAGES = tf.cast(np.array(IMAGES)/255.,dtype="float32")
LABELS = tf.cast(LABELS,dtype="float32")

In [ ]:
model.predict(IMAGES[10:11])

In [ ]:
LABELS.shape

In [ ]:
IMAGES.shape

In [ ]:
pred = model.predict(IMAGES[0:1])

In [ ]:
pred

In [ ]:
label = np.expand_dims(LABELS[0], axis=0)

In [ ]:
import tensorflow as tf
label = tf.cast(label,dtype="float32")

In [ ]:
label

In [ ]:
pred

In [ ]:
iou_loss(LABELS[0],LABELS[0])

In [ ]:
prediction = model.predict(IMAGES[0:1])

In [ ]:
prediction.shape

In [ ]:
type(LABELS[0])

In [ ]:
model.fit(x=IMAGES, y=LABELS, batch_size=1, epochs=20)

In [ ]:
LABELS[15]

In [ ]:
def predict_img(image):
    output = model.predict(image)
    box_pred = output[..., :4]
    box_pred = K.sigmoid(box_pred)
    
    conf_pred = output[..., 4:5]
    conf_pred = K.sigmoid(conf_pred)
    
    class_pred = output[..., 5:8]
    class_pred = K.softmax(class_pred)
    
    out = K.concatenate((box_pred, conf_pred),axis=3)
    out = K.concatenate((out, class_pred),axis=3)
    
    return out
    
    ###### PREDICTION FUNCTION WROTE ###########

In [ ]:
prediction = predict_img(IMAGES[10:11])

In [ ]:
prediction.shape

In [ ]:
prediction

In [ ]:
prediction

In [ ]:
predict_class = prediction[..., 5:8]  # 1 * 7 * 7 * 20
predict_trust = prediction[..., 4:5]  # 1 * 7 * 7 * 2
predict_box = prediction[..., :4]  # 1 * 7 * 7 * 8

predict_class = np.reshape(predict_class, [4, 4, 1, 3])
predict_trust = np.reshape(predict_trust, [4, 4, 1, 1])
predict_box = np.reshape(predict_box, [4, 4, 1, 4])

predict_scores = predict_class * predict_trust  # 7 * 7 * 2 * 20
#print(predict_scores)

box_classes = np.argmax(predict_scores, axis=-1)  # 7 * 7 * 2
box_class_scores = np.max(predict_scores, axis=-1)  # 7 * 7 * 2
best_box_class_scores = np.max(box_class_scores, axis=-1, keepdims=True)  # 7 * 7 * 1

box_mask = box_class_scores >= best_box_class_scores  # ? * 7 * 7 * 2

filter_mask = box_class_scores >= 0.1  # 7 * 7 * 2
filter_mask *= box_mask  # 7 * 7 * 2

filter_mask = np.expand_dims(filter_mask, axis=-1)  # 7 * 7 * 2 * 1


predict_scores *= filter_mask  # 7 * 7 * 2 * 20
pred = predict_box.copy()
pred *= filter_mask  # 7 * 7 * 2 * 4

box_classes = np.expand_dims(box_classes, axis=-1)
box_classes *= filter_mask  # 7 * 7 * 2 * 1

#box_xy, box_wh = yolo_head(predict_box)  # 7 * 7 * 2 * 2
box_xy_min, box_xy_max = xywh2minmax(128*pred[..., :2],128*pred[..., 2:4])  # 7 * 7 * 2 * 2

In [ ]:
pred_t = predict_trust.copy()
pred_t *= filter_mask  # 7 * 7 * 2 * 1
nms_mask = np.zeros_like(filter_mask)  # 7 * 7 * 2 * 1
predict_trust_max = np.max(pred_t)  # 找到置信度最高的框
max_i = max_j = max_k = 0
while predict_trust_max > 0:
    for i in range(nms_mask.shape[0]):
        for j in range(nms_mask.shape[1]):
            for k in range(nms_mask.shape[2]):
                if predict_trust[i, j, k, 0] == predict_trust_max:
                    nms_mask[i, j, k, 0] = 1
                    filter_mask[i, j, k, 0] = 0
                    max_i = i
                    max_j = j
                    max_k = k
    for i in range(nms_mask.shape[0]):
        for j in range(nms_mask.shape[1]):
            for k in range(nms_mask.shape[2]):
                if filter_mask[i, j, k, 0] == 1:
                    iou_score = iou(box_xy_min[max_i, max_j, max_k, :],
                                    box_xy_max[max_i, max_j, max_k, :],
                                    box_xy_min[i, j, k, :],
                                    box_xy_max[i, j, k, :])
                    if iou_score > 0.2:
                        filter_mask[i, j, k, 0] = 0
    pred_t *= filter_mask  # 7 * 7 * 2 * 1
    predict_trust_max = np.max(pred_t)  # 找到置信度最高的框

box_xy_min *= nms_mask
box_xy_max *= nms_mask

In [ ]:
box_xy_min

In [ ]:
classes_name=["human","static","ignore"]

In [ ]:
detect_shape = filter_mask.shape

for i in range(detect_shape[0]):
    for j in range(detect_shape[1]):
        for k in range(detect_shape[2]):
            if nms_mask[i, j, k, 0]:
                cv2.rectangle(image, (int(box_xy_min[i, j, k, 0]), int(box_xy_min[i, j, k, 1])),
                                (int(box_xy_max[i, j, k, 0]), int(box_xy_max[i, j, k, 1])),(0, 0, 255))
                cv2.putText(image, classes_name[box_classes[i, j, k, 0]],
                            (int(box_xy_min[i, j, k, 0]), int(box_xy_min[i, j, k, 1])),1, 1, (0, 0, 255))

In [ ]:
plt.imshow(image)

In [ ]:
pred

In [ ]:
prediction

In [ ]:
LABELS[10]

In [ ]:
xywh2minmax(128*np.array([0.229143, 0.585366]),128*np.array([0.195429, 0.411585]))

In [ ]:
image = np.array(IMAGES[10])*255

start_point = (17, 48)
end_point = ( 42 , 101)
color = (255, 0, 0)
thickness = 2

image = cv2.rectangle(image, start_point, end_point, color, thickness)

plt.imshow(image)

In [ ]:
np.argmax(x,axis=3)

In [ ]:
np.argmax(np.array([0.3333226 , 0.33326322, 0.33341417]))

In [ ]:
def non_max_supression():
    ###### NMS FUNCTION WROTE ###############

In [ ]:
def iou_loss(y_true, y_pred):
    # iou loss for bounding box prediction
    # input must be as [x1, y1, x2, y2]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] - K.transpose(y_true)[0] + 1) * K.abs(K.transpose(y_true)[3] - K.transpose(y_true)[1] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] - K.transpose(y_pred)[0] + 1) * K.abs(K.transpose(y_pred)[3] - K.transpose(y_pred)[1] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2], K.transpose(y_pred)[2])
    overlap_3 = K.minimum(K.transpose(y_true)[3], K.transpose(y_pred)[3])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    # loss for the iou value
    iou_loss = -K.log(iou)

    return iou_loss

In [ ]:
import tensorflow.keras.backend as K
K.sigmoid((0.25))

In [ ]:
label = LABELS[0][..., :4]

In [ ]:
label = LABELS[0]

In [ ]:
label.shape

In [ ]:
label = LABELS[0]

In [ ]:
label[..., 4:5].shape

In [ ]:
label[..., 5:8]

In [ ]:
label = np.expand_dims(label, axis=0)

In [ ]:
label[..., :2]

In [ ]:
deneme = np.random.rand(4,4,4)

In [ ]:
xxx = K.sigmoid(label[..., :4])

In [ ]:
K.softmax(label[..., 5:8])

In [ ]:
box_loss = K.square(label[..., :4] - xxx)

In [ ]:
K.sum(box_loss)

In [ ]:
bb_intersection_over_union([16.822848, 77.853632, 22.820544,  94.438976],[17,50,23,95])

In [ ]:
xywh2minmax(128*label[..., :2],128*label[..., 2:4])[0]

In [ ]:
xywh2minmax(128*label[..., :2],128*label[..., 2:4])[1]

In [ ]:
aa=xywh2minmax(128*label[..., :2],128*label[..., 2:4])[0]

In [ ]:
bb=xywh2minmax(128*label[..., :2],128*label[..., 2:4])[1]

In [ ]:
tt=np.concatenate((aa,bb),axis=3)

In [ ]:
tt.shape

In [ ]:
iou_loss(tt,)

In [ ]:
aa.shape

In [ ]:
for i in range(4):
    for j in range(4):
        print(bb_intersection_over_union(np.concatenate((aa,bb),axis=3)[0][i][j],np.concatenate((aa,bb),axis=3)[0][i][j]))

In [ ]:
bb_intersection_over_union(np.concatenate((aa,bb),axis=3),np.concatenate((aa,bb),axis=3))

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model


def model_tiny_yolov1(inputs):
    x = Conv2D(2, (3, 3), padding='same', name='convolutional_0', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(inputs)
    x = BatchNormalization(name='bnconvolutional_0', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = Conv2D(4, (3, 3), padding='same', name='convolutional_1', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_1', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), padding='same', name='convolutional_2', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_2', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), padding='same', name='convolutional_3', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_3', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = Conv2D(32, (3, 3), padding='same', name='convolutional_4', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_4', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = Conv2D(64, (3, 3), padding='same', name='convolutional_5', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_5', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = Conv2D(128, (3, 3), padding='same', name='convolutional_6', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_6', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(64, (3, 3), padding='same', name='convolutional_7', use_bias=False,
               kernel_regularizer=l2(5e-4), trainable=False)(x)
    x = BatchNormalization(name='bnconvolutional_7', trainable=False)(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Flatten()(x)
    x = Dense(128, activation='linear', name='connected_0')(x)
    x = Reshape((4, 4, 8))(x)
    #outputs = Yolo_Reshape((4, 4, 7))(x)

    return x

In [ ]:
input_shape = (1, 128, 128, 1)

inputs = Input(input_shape[1:4])

model = model_tiny_yolov1(inputs)

In [ ]:
outputs = model_tiny_yolov1(inputs)
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
import tensorflow.keras.backend as K


def xywh2minmax(xy, wh):
    xy_min = xy - wh / 2
    xy_max = xy + wh / 2

    return xy_min, xy_max


def iou(pred_mins, pred_maxes, true_mins, true_maxes):
    intersect_mins = K.maximum(pred_mins, true_mins)
    intersect_maxes = K.minimum(pred_maxes, true_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    pred_wh = pred_maxes - pred_mins
    true_wh = true_maxes - true_mins
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]
    true_areas = true_wh[..., 0] * true_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores = intersect_areas / union_areas

    return iou_scores


def yolo_head(feats):
    # Dynamic implementation of conv dims for fully convolutional model.
    conv_dims = K.shape(feats)[1:3]  # assuming channels last
    # In YOLO the height index is the inner most iteration.
    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])

    # TODO: Repeat_elements and tf.split doesn't support dynamic splits.
    # conv_width_index = K.repeat_elements(conv_width_index, conv_dims[1], axis=0)
    conv_width_index = K.tile(
        K.expand_dims(conv_width_index, 0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_index, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(feats))

    conv_dims = K.cast(K.reshape(conv_dims, [1, 1, 1, 1, 2]), K.dtype(feats))

    box_xy = (feats[..., :2] + conv_index) / conv_dims * 448
    box_wh = feats[..., 2:4] * 448

    return box_xy, box_wh

##############
### ONLY PROBLEM IS RESPONSE MASK TRY TO UNDERSTAND THAT REALLY WELL PLEASE

def yolo_loss(y_true, y_pred):
    label_class = y_true[..., :20]  # ? * 7 * 7 * 20
    label_box = y_true[..., 20:24]  # ? * 7 * 7 * 4
    response_mask = y_true[..., 24]  # ? * 7 * 7
    response_mask = K.expand_dims(response_mask)  # ? * 7 * 7 * 1

    predict_class = y_pred[..., :20]  # ? * 7 * 7 * 20
    predict_trust = y_pred[..., 20:22]  # ? * 7 * 7 * 2
    predict_box = y_pred[..., 22:]  # ? * 7 * 7 * 8

    _label_box = K.reshape(label_box, [-1, 7, 7, 1, 4])
    _predict_box = K.reshape(predict_box, [-1, 7, 7, 2, 4])

    label_xy, label_wh = yolo_head(_label_box)  # ? * 7 * 7 * 1 * 2, ? * 7 * 7 * 1 * 2
    label_xy = K.expand_dims(label_xy, 3)  # ? * 7 * 7 * 1 * 1 * 2
    label_wh = K.expand_dims(label_wh, 3)  # ? * 7 * 7 * 1 * 1 * 2
    label_xy_min, label_xy_max = xywh2minmax(label_xy, label_wh)  # ? * 7 * 7 * 1 * 1 * 2, ? * 7 * 7 * 1 * 1 * 2

    predict_xy, predict_wh = yolo_head(_predict_box)  # ? * 7 * 7 * 2 * 2, ? * 7 * 7 * 2 * 2
    predict_xy = K.expand_dims(predict_xy, 4)  # ? * 7 * 7 * 2 * 1 * 2
    predict_wh = K.expand_dims(predict_wh, 4)  # ? * 7 * 7 * 2 * 1 * 2
    predict_xy_min, predict_xy_max = xywh2minmax(predict_xy, predict_wh)  # ? * 7 * 7 * 2 * 1 * 2, ? * 7 * 7 * 2 * 1 * 2

    iou_scores = iou(predict_xy_min, predict_xy_max, label_xy_min, label_xy_max)  # ? * 7 * 7 * 2 * 1
    print(predict_xy_min.shape)
    best_ious = K.max(iou_scores, axis=4)  # ? * 7 * 7 * 2
    best_box = K.max(best_ious, axis=3, keepdims=True)  # ? * 7 * 7 * 1

    box_mask = K.cast(best_ious >= best_box, K.dtype(best_ious))  # ? * 7 * 7 * 2

    no_object_loss = 0.5 * (1 - box_mask * response_mask) * K.square(0 - predict_trust)
    object_loss = box_mask * response_mask * K.square(1 - predict_trust)
    confidence_loss = no_object_loss + object_loss
    confidence_loss = K.sum(confidence_loss)

    class_loss = response_mask * K.square(label_class - predict_class)
    class_loss = K.sum(class_loss)

    _label_box = K.reshape(label_box, [-1, 7, 7, 1, 4])
    _predict_box = K.reshape(predict_box, [-1, 7, 7, 2, 4])

    label_xy, label_wh = yolo_head(_label_box)  # ? * 7 * 7 * 1 * 2, ? * 7 * 7 * 1 * 2
    predict_xy, predict_wh = yolo_head(_predict_box)  # ? * 7 * 7 * 2 * 2, ? * 7 * 7 * 2 * 2

    box_mask = K.expand_dims(box_mask)
    response_mask = K.expand_dims(response_mask)

    box_loss = 5 * box_mask * response_mask * K.square((label_xy - predict_xy) / 448)
    box_loss += 5 * box_mask * response_mask * K.square((K.sqrt(label_wh) - K.sqrt(predict_wh)) / 448)
    box_loss = K.sum(box_loss)

    loss = confidence_loss + class_loss + box_loss

    return loss

In [ ]:
import tensorflow as tf

#tf.cast(
#    x, dtype=float64, name=None
#)

#y_true = label
#y_pred = pred

box_true = y_true[..., :4]
box_pred = y_pred[..., :4]
box_pred = K.sigmoid(box_pred)
    
xy_min_pred, xy_max_pred = xywh2minmax(128*y_pred[..., :2],128*y_pred[..., 2:4])
xy_min_true, xy_max_true = xywh2minmax(128*y_true[..., :2],128*y_true[..., 2:4])
print(xy_min_true.shape)

iou_scores = iou(xy_min_pred, xy_max_pred, xy_min_true, xy_max_true)

    
#boxes_pred = K.concatenate((xy_min_pred, xy_max_pred),axis=3)
#boxes_true = K.concatenate((xy_min_true, xy_max_true),axis=3)
    
    #print(boxes_true)
    #print(boxes_pred)
    
    #print(boxes_pred)
    #print(boxes_true)
#print(boxes_true)
#print(boxes_pred)
#K.sum(iou_loss(boxes_true,boxes_pred))

In [ ]:
iou_scores
iou_scores=K.expand_dims(iou_scores)
iou_scores=K.expand_dims(iou_scores)
iou_scores=K.expand_dims(iou_scores,axis=0)

In [ ]:
best_ious = K.max(iou_scores, axis=4)

In [ ]:
best_box = K.max(best_ious, axis=3, keepdims=True)

In [ ]:
best_box

In [ ]:
box_mask = K.cast(best_ious >= best_box, K.dtype(best_ious))

In [ ]:
box_mask

In [ ]:
iou_scores

In [ ]:
y_true[..., 5].shape

In [ ]:
y_true[2][0][24]

In [ ]:
y_true = np.random.rand(4,4,8)
y_pred = np.random.rand(4,4,8)

In [ ]:
yolo_loss(y_true,y_pred)